This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
#2 columns = 'FPGA' and 'TDC_CHANNEL'
#times : 'ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'
#BX_COUNTER : 25 ns
#ORBIT_CNT : increased every 'x' BX_COUNTER
#TDC_MEAS : 25/30 ns

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
import pandas as pd

filename = 'data_000637.txt'
#reading Nrows data
N_data = pd.read_csv(filename, nrows=10000)
print("Data of 10k rows\n", N_data)

#reading all data
data = pd.read_csv(filename)
print("\nAll of Data: \n", data)

Data of 10k rows
       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0        1     0          123  3869200167        2374        26
1        1     0          124  3869200167        2374        27
2        1     0           63  3869200167        2553        28
3        1     0           64  3869200167        2558        19
4        1     0           64  3869200167        2760        25
...    ...   ...          ...         ...         ...       ...
9995     1     0           61  3869200267         696        11
9996     1     0           60  3869200267         701         5
9997     1     0           59  3869200267         707        23
9998     1     0           63  3869200267         706        15
9999     1     0           49  3869200267         777        13

[10000 rows x 6 columns]

All of Data: 
          HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  3869200167        2374        26
1           1     0          124  3869

2\. Estimate the number of BX in a ORBIT (the value 'x').

In [2]:
import numpy as np
x = np.max(data['BX_COUNTER'] + 1)
print("\nNumber of BX(x): ", x)


Number of BX(x):  3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [3]:
#calculating time of 1st row, last row, and the difference
time_1st_row = data.loc[data.index[0], 'ORBIT_CNT']*3564*25 + data.loc[data.index[0],'BX_COUNTER']*25 + data.loc[data.index[0],'TDC_MEAS']*25/30
time_last_row = data.loc[data.index[-1], 'ORBIT_CNT']*3564*25 + data.loc[data.index[-1],'BX_COUNTER']*25 + data.loc[data.index[-1],'TDC_MEAS']*25/30
duration = time_last_row - time_1st_row

#converting to seconds, minutes, and hours
seconds = duration*0.000000001
minutes = seconds/60
hours = minutes/60


print("Duration in getting the data: \n", duration, " nanoseconds, ", seconds, " seconds, ", minutes, " minutes, ", hours, " hours ")

Duration in getting the data: 
 980416093.3125  nanoseconds,  0.9804160933125  seconds,  0.016340268221875  minutes,  0.00027233780369791664  hours 


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [4]:
data['Time in ns'] = data['ORBIT_CNT']*3564*25 + data['BX_COUNTER']*25 + data['TDC_MEAS']*25/30
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time in ns
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,1,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,3.447467e+14
1310716,1,1,4,3869211171,763,11,3.447467e+14
1310717,1,0,64,3869211171,764,0,3.447467e+14
1310718,1,0,139,3869211171,769,0,3.447467e+14


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [5]:
noise = data.groupby(by=["TDC_CHANNEL"]).size().sort_values(ascending = False).head(3)
noise

#DataFrame.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)

TDC_CHANNEL
139    108059
64      66020
63      64642
dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [6]:
#counting number of non-empty orbits
non_empty_orbits = data.groupby('ORBIT_CNT')
print("Number of non-empty orbits: ", len(non_empty_orbits))

#counting number of TDC_CHANNEL=139
tdc_channel139 = data[data['TDC_CHANNEL'] == 139].groupby('ORBIT_CNT')
print("Number of unique orbits where TDC_CHANNEL is equal to 139: ", len(tdc_channel139))

Number of non-empty orbits:  11001
Number of unique orbits where TDC_CHANNEL is equal to 139:  10976


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel